In [68]:
import requests
from pandas.io.json import json_normalize
import os
import pandas as pd
import time
import math
from datetime import datetime
import numpy as np
import sqlalchemy 
import psycopg2
import smtplib

In [69]:
company = 'GAME_STOP'
keyword = '"Game Stop"'
date_str = '2020-10-01'
date = datetime.strptime(date_str, '%Y-%m-%d')
date

datetime.datetime(2020, 10, 1, 0, 0)

In [79]:
class PushShiftPull():
    def _init_(keyword,last_comment,last_submmision,company):
        self.company = company
        self.keyword = keyword
        self.last_comment = last_comment
        self.last_submission = last_submission
    def create_directories(self):
        def check_create(directory):
            if not os.path.exists(directory):
                os.makedirs(directory)
        #Topline
        self.directory = os.path.join(os.getcwd(),'Reddit_Content',self.company,self.keyword)
        check_create(self.directory)
        #Comments
        self.directory_comments = os.path.join(self.directory,'comments')
        check_create(self.directory_comments)
        #Posts
        self.directory_posts = os.path.join(self.directory,'posts')
        check_create(self.directory_posts)
    def downloadFromUrl_comment(self):
        last_comment_timestamp = int(self.last_comment.timestamp())
        #print(last_comment_timestamp)
        #url = "https://api.pushshift.io/reddit/comment/search?q={}&after={}&limit=1000&sort=desc&before=".format(self.keyword,self.comment_max)
        url = "https://api.pushshift.io/reddit/comment/search?q={}&limit=1000&subreddit=wallstreetbets&sort=desc&after={}&before=".format(self.keyword,last_comment_timestamp )
        print(url)
        count = 0
        previous_epoch = int(datetime.utcnow().timestamp())
        dfs = []
        while True:
            #new_url = url+str(previous_epoch)
            new_url = url.format(self.keyword,last_comment_timestamp)+str(previous_epoch)
            print(new_url)
            json = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
            try:
                json_data = json.json()
            except:
                print('failed!')
                break
            if 'data' not in json_data:
                break
            objects = json_data['data']
            if len(objects) == 0:
                break
            print('no break')
            for object in objects:
                previous_epoch = object['created_utc'] - 1
                count += 1
                try:
                    if count == 1:
                        self.comment_df = json_normalize(object)
                    else:
                        df = json_normalize(object)
                        self.comment_df = pd.concat([self.comment_df,df],axis = 0,ignore_index = True,sort=False)
                except Exception as err:
                    print(f"Couldn't print comment: https://www.reddit.com{object['permalink']}")
                    print(traceback.format_exc())
            print("Saved {} comments through {}".format(count, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))
        try:
            #self.comment_df = pd.concat(dfs,axis = 0,ignore_index = True,sort=False)
            self.comment_df['created_utc'] = pd.to_datetime(self.comment_df['created_utc'],unit='s')
            self.comment_df['type'] = 'comment'
            columns = ['author','author_fullname','body','created_utc','id','link_id','parent_id','permalink','score','subreddit','subreddit_id','subreddit_name_prefixed','subreddit_type','total_awards_received','type']
            self.comment_df = self.comment_df.reindex(columns = columns)
            #self.comment_df.to_csv(self.filename_comments,index = False)
        except:
            print('Not comments for ' + str(self.keyword))
    def downloadFromUrl_submission(self):
        #url = "https://api.pushshift.io/reddit/submission/search?q={}&after={}&limit=1000&sort=desc&author=&before=".format(self.keyword,self.submission_max)
        #print(url)
        last_sumbission = int(self.last_submission.timestamp())
        url = "https://api.pushshift.io/reddit/submission/search?q={}&limit=1000&subreddit=wallstreetbets&sort=desc&author=&after={}&before=".format(self.keyword,last_sumbission)
        count = 0
        previous_epoch = int(datetime.utcnow().timestamp())
        dfs = []
        while True:
            new_url = url.format(self.keyword,last_sumbission)+str(previous_epoch)
            json = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
            print('got json')
            try:
                json_data = json.json()
            except:
                print('failed!')
                break
            if 'data' not in json_data:
                break
            objects = json_data['data']
            if len(objects) == 0:
                break
            for object in objects:
                previous_epoch = object['created_utc'] - 1
                count += 1
                #if object['is_self']:
                #    if 'selftext' not in object:
                #        continue
                try:
                    if count == 1:
                        self.submission_df = json_normalize(object)
                    else:
                        df = json_normalize(object)
                        self.submission_df = pd.concat([self.submission_df,df],axis = 0,ignore_index = True,sort=False)
                except Exception as err:
                    print(f"Couldn't print post: {object['url']}")
                    print(traceback.format_exc())
            print("Saved {} submissions through {}".format(count, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))
        try:
            self.submission_df['created_utc'] = pd.to_datetime(self.submission_df['created_utc'],unit='s')
            self.submission_df['type'] = 'submission'
            columns = ['author','author_fullname','created_utc','full_link','id','is_meta','is_original_content','is_reddit_media_domain','is_robot_indexable','is_self','is_video','media_only','num_comments','num_crossposts','permalink','score','selftext','subreddit','subreddit_id','subreddit_subscribers','subreddit_type','thumbnail','title','url','crosspost_parent','promoted','media.type']
            self.submission_df = self.submission_df.reindex(columns = columns)
        except:
            print('No DF for Submissions!')
        #self.submission_df.to_csv(self.filename_submission,index = False)
    def create_files(self):
        #comments
        print('Saving Comments')
        cdate_min = str(np.min(self.comment_df['created_utc']))
        cdate_max = str(np.max(self.comment_df['created_utc']))
        comment_stuff = [self.company,self.keyword,'comments',cdate_min,cdate_max]
        comments_files = '-'.join(comment_stuff )
        self.comment_df.to_csv(os.path.join(self.directory_comments,comments_files),index = False)
        print('Saved Comments')
        #Posts
        pdate_min = str(np.min(self.submission_df['created_utc']))
        pdate_max = str(np.max(self.submission_df['created_utc']))
        submission_stuff = [self.company,self.keyword,'posts',pdate_min,pdate_max]
        submission_file = '-'.join(submission_stuff)
        self.submission_df.to_csv(os.path.join(self.directory_posts,submission_file),index = False)

In [80]:
def Run_PushShift(keyword,company,last_comment,last_submission):
    print(keyword)
    print(last_comment)
    print(last_submission)
    #Associate Variables
    Test = PushShiftPull()
    Test.company = company
    Test.keyword = keyword
    Test.last_comment = last_comment
    Test.last_submission = last_submission
    #Run Code
    Test.create_directories()
    Test.downloadFromUrl_comment()
    Test.downloadFromUrl_submission()
    Test.create_files()
    return(Test)

In [81]:
Avery = Run_PushShift(keyword,company,date,date)

"Game Stop"
2020-10-01 00:00:00
2020-10-01 00:00:00
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611727067
no break


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Saved 100 comments through 2021-01-22
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611350001
no break
Saved 200 comments through 2021-01-21
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611271597
no break
Saved 300 comments through 2021-01-19
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1611092092
no break
Saved 400 comments through 2021-01-16
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1610816177
no break
Saved 500 comments through 2021-01-14
https://api.pushshift.io/reddit/comment/search?q="Game Stop"&limit=1000&subreddit=wallstreetbets&sort=desc&after=1601524800&before=1610640916
no break
Saved 600 comments through 2021-01-01
https://api.pushs

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:97: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Saved 100 submissions through 2020-10-18
got json
Saved 102 submissions through 2020-10-12
got json


In [59]:
Avery.comment_df

,author,author_fullname,body,created_utc,id,link_id,parent_id,permalink,score,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,total_awards_received,type
0,Fireball5-,t2_2utjiar2,GAME STOP,2021-01-24 12:02:29,gkjjtx1,t3_l3y4mp,t1_gkjj9c2,/r/wallstreetbets/comments/l3y4mp/gme_squeeze_...,20,wallstreetbets,t5_2th52,NaN,NaN,0,comment
1,kasperdodo,t2_58we8puy,bruh why y'all investing in game stop all of s...,2021-01-24 11:36:12,gkjfuq6,t3_l3onms,t3_l3onms,/r/wallstreetbets/comments/l3onms/ryan_cohen_d...,-2,wallstreetbets,t5_2th52,NaN,NaN,0,comment
2,kvla1,t2_54iayt2b,I agree on the short squeeze. Applying chewy s...,2021-01-24 11:34:54,gkjfl5w,t3_l2wx2p,t1_gkjewrv,/r/wallstreetbets/comments/l2wx2p/weekend_disc...,2,wallstreetbets,t5_2th52,NaN,NaN,0,comment
3,chinaisasshole,t2_33qnoq33,If this going to be the next game stop plz tel...,2021-01-24 08:04:50,gkih07h,t3_l3nmk8,t3_l3nmk8,/r/wallstreetbets/comments/l3nmk8/wtf_is_going...,0,wallstreetbets,t5_2th52,NaN,NaN,0,comment
4,Grymninja,t2_9bcif,Yeah well every game stop OTM weekly call for ...,2021-01-24 06:41:06,gki5cvv,t3_l3re7i,t1_gkhuxow,/r/wallstreetbets/comments/l3re7i/i_present_to...,3,wallstreetbets,t5_2th52,NaN,NaN,0,comment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,by_any_other_names,t2_bdshoi,If your in game stop you deserve to lose money,2020-10-08 17:09:53,g84jprt,t3_j7agfu,t1_g84j83w,/r/wallstreetbets/comments/j7agfu/daily_discus...,1,wallstreetbets,t5_2th52,NaN,NaN,0,comment
1006,omgitsdoxa,t2_nqedt,Game stop?,2020-10-07 17:29:38,g80t1n8,t3_j6uoi5,t1_g80q16l,/r/wallstreetbets/comments/j6uoi5/in_search_of...,1,wallstreetbets,t5_2th52,NaN,NaN,0,comment
1007,NeverOddOreveN69,t2_774neusp,Shoulda bought game stop!\n\nMuch love!,2020-10-06 20:46:31,g7xpckc,t3_j6ccxn,t3_j6ccxn,/r/wallstreetbets/comments/j6ccxn/how_i_earned...,2,wallstreetbets,t5_2th52,NaN,NaN,0,comment
1008,Fontec,t2_jmfk7,positions spy puts and game stop covered puts,2020-10-06 17:04:00,g7wtt6f,t3_j68vng,t3_j68vng,/r/wallstreetbets/comments/j68vng/tracking_the...,1,wallstreetbets,t5_2th52,NaN,NaN,0,comment
